In [1]:
import sys
!jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-threejs
!jupyter nbextension install --py --symlink --sys-prefix pythreejs
!jupyter nbextension enable --py --sys-prefix pythreejs
#!{sys.executable} -m pip install pythreejs


(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/home/runner/workspace/.pythonlibs/lib/python3.11/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/home/runner/workspace/.pythonlibs/lib/python3.11/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/home/runner/workspace/.pythonlibs/lib/python3.11/site-packages/jupyterlab/debuglog.py:56: UserWarning: See the log file for details: /tmp/jupyterlab-debug-x2xua

In [1]:
# Import the Doyle spiral module
import sys
sys.path.insert(0, 'src')

from doyle_spiral import spiral_ui

# Run the interactive UI
spiral_ui()

## Good settings

p = 8, q = 16
mode = "angular"
n_gaps = 2

p = 16, q = 32
mode = "angular"
n_gaps = 2